In [14]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from zemberek_python.main_libs import nltk_download, ZemberekTool
from kefir.predication import personal, inferential
from kefir.subject import Person
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)

In [15]:
morphology = TurkishMorphology.create_with_defaults()
normalizer = TurkishSentenceNormalizer(morphology)

2021-01-18 15:28:24,074 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 13.496797800064087



In [16]:
zemberekTool = ZemberekTool()

In [17]:
dolaylı_tumlec_ekleri = ["ISIM_CIKMA_DEN","ISIM_KALMA_DE","ISIM_CIKMA_DE","ISIM_YONELME_E"]

### Cumlenin ögelerini bulmamıza yardımcı olacak sonlu durum otamatasını tasarlayalım.

In [10]:
from automata.fa.nfa import NFA
nfa = NFA(
    states={'q_baslangic', 'q_ozne', 'q_zarf', 'q_yuklem', 'q_nesne', 'q_tumlec', 'q_edat'},
    input_symbols={'a', 'b', 'c', 'd' , 'e', 'f' ,'h'},
    transitions={
        'q_baslangic': {'a': {'q_ozne'}, 'b': {'q_zarf'}, 'c': {'q_yuklem'}},
        'q_ozne': {'d': {'q_nesne'}, 'e': {'q_tumlec'},'f': {'q_edat'}, 'h': {'q_ozne'}},
        'q_nesne': {'h': {'q_nesne'}},
        'q_tumlec': {'h': {'q_tumlec'}},
        'q_edat': {'h': {'q_edat'}},      
        'q_zarf': {'h': {'q_zarf'}},
        'q_yuklem': {'h': {'q_yuklem'}}
    },
    initial_state='q_baslangic',
    final_states={'q_ozne', 'q_zarf', 'q_yuklem', 'q_nesne', 'q_tumlec', 'q_edat'}
)

### Kelimelerin kök ve eklerinin türlerini bularak otomatanın alfabesine çeviren fonksiyon

In [11]:
oge_list = []
def oge_bulucu(word):
    if zemberekTool.ogelere_ayir(word) is None:
            oge_list.append("a")
            oge_list.append(".")
    else:         
        for ek_turu in zemberekTool.ogelere_ayir(word)["ekler"].replace(" ","").split("+"):

            if 'ISIM_KOK' == ek_turu:
                oge_list.append("a")
            elif 'ZAMAN_KOK' == ek_turu:
                oge_list.append("b")
            elif 'FIIL_KOK' == ek_turu:
                oge_list.append("c")
            elif 'ISIM_BELIRTME_I' == ek_turu or "ISIM_TAMLAMA_I" == ek_turu:
                oge_list.append("d")
            elif ek_turu in dolaylı_tumlec_ekleri:
                oge_list.append("e")
            elif 'ISIM_BIRLIKTELIK_LE' in zemberekTool.ogelere_ayir(word)["ekler"].replace(" ","").split("+"):
                oge_list.append("f")   

            else:
                oge_list.append("h")
            #[oge_list.append("e") if ek in dolaylı_tumlec_ekleri else oge_list.append("h") for ek in zemberekTool.ogelere_ayir(word)["ekler"].replace(" ","").split("+")]
        oge_list.append(".")

### Son olarak cümlenin ögelerini bulmamızı sağlayan fonksiyon.

In [12]:
def cumlenin_ogeleri(text):
    normalize_text = normalizer.normalize(text)
    wordTokenize = word_tokenize(normalize_text)

    for word in wordTokenize:
        oge_bulucu(word)
    ogeler =[]
    ogeler2=""
    for oge in oge_list:
        if oge == ".":
            ogeler.append(ogeler2)
            ogeler2=""
        else:
            ogeler2=ogeler2+oge
        print(ogeler," ",ogeler2)
    print("--------------------------")
    for i in ogeler:
        print(nfa.read_input(i))
    print("--------------------------")
    print(oge_list,wordTokenize)

### Örnek cümlelerde test aşaması

In [18]:
oge_list = []
cumlenin_ogeleri("babam dün yemeğe gitti")

[]   a
[]   ah
['ah']   
['ah']   b
['ah', 'b']   
['ah', 'b']   a
['ah', 'b']   ae
['ah', 'b', 'ae']   
['ah', 'b', 'ae']   c
['ah', 'b', 'ae']   ch
['ah', 'b', 'ae', 'ch']   
--------------------------
{'q_ozne'}
{'q_zarf'}
{'q_tumlec'}
{'q_yuklem'}
--------------------------
['a', 'h', '.', 'b', '.', 'a', 'e', '.', 'c', 'h', '.'] ['babam', 'dün', 'yemeğe', 'gitti']


In [22]:
oge_list = []
cumlenin_ogeleri("mehmet okulu sevmiyor")

[]   a
['a']   
['a']   a
['a', 'a']   
['a', 'a']   c
['a', 'a']   ch
['a', 'a']   chh
['a', 'a', 'chh']   
--------------------------
{'q_ozne'}
{'q_ozne'}
{'q_yuklem'}
--------------------------
['a', '.', 'a', '.', 'c', 'h', 'h', '.'] ['mehmet', 'okul', 'sevmiyor']


In [23]:
oge_list = []
cumlenin_ogeleri("ahmet okula arabayla gitti")

[]   a
['a']   
['a']   a
['a']   ae
['a', 'ae']   
['a', 'ae']   a
['a', 'ae']   af
['a', 'ae', 'af']   
['a', 'ae', 'af']   c
['a', 'ae', 'af']   ch
['a', 'ae', 'af', 'ch']   
--------------------------
{'q_ozne'}
{'q_tumlec'}
{'q_edat'}
{'q_yuklem'}
--------------------------
['a', '.', 'a', 'e', '.', 'a', 'f', '.', 'c', 'h', '.'] ['ahmet', 'okula', 'arabayla', 'gitti']


In [24]:
oge_list = []
cumlenin_ogeleri("halil yemeği çatalla yer")

[]   a
['a']   
['a']   a
['a']   ad
['a', 'ad']   
['a', 'ad']   a
['a', 'ad']   af
['a', 'ad', 'af']   
['a', 'ad', 'af']   c
['a', 'ad', 'af', 'c']   
--------------------------
{'q_ozne'}
{'q_nesne'}
{'q_edat'}
{'q_yuklem'}
--------------------------
['a', '.', 'a', 'd', '.', 'a', 'f', '.', 'c', '.'] ['halil', 'yemeği', 'çatalla', 'yer']
